In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport bitnotes
%aimport watch_point
%aimport cluster
%aimport constants
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from bitnotes import BitNotes
from pprint import pprint
import math
import constants as const

### Load phrase base

In [2]:
import pickle

def load_phrase_base(file_name: str) -> (dict, list):
    with open(file_name, 'rb') as f:
        data = pickle.load(f)
        phrase_base = data.get('phrase_base', {})
        marks = data.get('marks', {})
        return phrase_base, marks
    
# phrase_base, marks = load_phrase_base('./data/texts/prob_text_base.pickle')
# phrase_base, marks = load_phrase_base('./data/texts/lang_base.pickle')
# phrase_base, marks = load_phrase_base('./data/texts/monotone_base.pickle')
phrase_base, marks = load_phrase_base('./data/texts/dna_base.pickle')

In [3]:
marks

{(0,): 'abcd',
 (1,): 'cdef',
 (2,): 'klmn',
 (3,): 'ijhg',
 (4,): 'oprs',
 (5,): 'xyzw',
 (6,): 'quva',
 (7,): 'kiox',
 (8,): 'nghp'}

### Draw notations

In [ ]:
from ipythonblocks import BlockGrid

def draw_notation(notation: np.array):
    bit_grid = BlockGrid(len(notation), 1, fill=(17, 41, 129))
    for block in range(bit_grid.width):
        color = bit_grid[0, block]
        if notation[block]:
            bit_grid[0, block] = (244, 195, 173)
    bit_grid.lines_on = False
    bit_grid.show()

def draw_note_notations(notes, note_idx: int):
    for i in range(len(notes[note_idx])):
        draw_notation(notes.note_notation_as_bits(note_idx, i))

In [ ]:
abc_notes = BitNotes(note_count=26, 
                     notation_count=5, 
                     active_bits=8, 
                     bit_count=255)

key = list(phrase_base.keys())[0]
phrase = phrase_base[key][0]
bit_chord = abc_notes.phrase_chord(phrase)
draw_notation(abc_notes.notation_as_bits(bit_chord))
print(phrase)
for note_idx, notation_idx in phrase:
    notation = abc_notes.note_notation_as_bits(note_idx, notation_idx)
    draw_notation(notation)

In [ ]:
feeder = PhraseFeeder(phrase_base, marks)    

In [ ]:
phrases, output_bits = feeder.take_phrase_bunch()
print(output_bits, marks[output_bits])
print(phrases)

### Learn model

In [4]:
def feed_phrase_batch(cxnn: ContextNN, feeder: PhraseFeeder, bit_notes: BitNotes, count=200):
    phrases, output_bits = feeder.take_phrase_batch(count=200, random_bit_key=False)
    mark = feeder.marks[output_bits]    
    output_bits = set(output_bits)
    for phrase in phrases:
        bit_chord = bit_notes.phrase_chord_as_bits(phrase)
        cxnn.receive_bits(input_bits=bit_chord, output_bits=output_bits)
    return mark    

def feed_n_batches(cxnn: ContextNN, feeder: PhraseFeeder, bit_notes: BitNotes, n=10, batch_size=200):
    for i in range(n):
        mark = feed_phrase_batch(cxnn, feeder, bit_notes, count=batch_size)
        print(f'batch {i+1}/{n} {mark}')
    print(f'clusters: {cxnn.cluster_count()}')
        
def learn_cycle(cxnn: ContextNN, 
                feeder: PhraseFeeder, 
                bit_notes: BitNotes,
                batch_size=200,
                consolidate=True,
                reduce=True,
                n_accumulate_batches=30,
                n_consolidate_batches=50,
                reduce_min_component=0.2,
                reduce_min_activations=100):
    print('\naccumulating clusters...')
    cxnn.state = const.STATE_ACCUMULATE
    feed_n_batches(cxnn, feeder, bit_notes, n=n_accumulate_batches, batch_size=batch_size)
    
    if consolidate:
        print('\nconsolidating clusters...')
        cxnn.state = const.STATE_CONSOLIDATE
        feed_n_batches(cxnn, feeder, bit_notes, n=n_consolidate_batches, batch_size=batch_size)
    
    if reduce:
        print('\nreducing clusters...')
        cxnn.reduce_clusters(min_component=reduce_min_component, 
                             min_activations=reduce_min_activations,
                             trim=True,
                             remain_parts=3,
                             clear_stats=True,
                             consolidate=True,
                             amnesty=True)
        print(f'clusters: {cxnn.cluster_count()}')
    

In [8]:
notes = bitnotes.BitNotes(note_count=26, 
                          notation_count=5, 
                          active_bits=8, 
                          bit_count=256)

feeder = phrase_feeder.PhraseFeeder(phrase_base, marks)  

cxnn = context_nn.ContextNN(input_bit_count=256,
                            output_bit_count=len(marks.keys()),
                            watch_point_count=2000,
                            watch_bit_count=32,
                            cluster_make_threshold=7,
                            cluster_activate_threshold=5,
                            bit_notes=notes,
                            data_marks=marks)

In [9]:
%%time
for n in range(5):
    print(f'\nCycle {n+1}')
    learn_cycle(cxnn, 
                feeder, 
                notes,
                batch_size=200,
                consolidate=True,
                reduce=True,
                n_accumulate_batches=32,
                n_consolidate_batches=16,
                reduce_min_component=0.15,
                reduce_min_activations=100)


Cycle 1

accumulating clusters...
batch 1/32 abcd
batch 2/32 cdef
batch 3/32 klmn
batch 4/32 ijhg
batch 5/32 oprs
batch 6/32 xyzw
batch 7/32 quva
batch 8/32 kiox
batch 9/32 nghp
batch 10/32 abcd
batch 11/32 cdef
batch 12/32 klmn
batch 13/32 ijhg
batch 14/32 oprs
batch 15/32 xyzw
batch 16/32 quva
batch 17/32 kiox
batch 18/32 nghp
batch 19/32 abcd
batch 20/32 cdef
batch 21/32 klmn
batch 22/32 ijhg
batch 23/32 oprs
batch 24/32 xyzw
batch 25/32 quva
batch 26/32 kiox
batch 27/32 nghp
batch 28/32 abcd
batch 29/32 cdef
batch 30/32 klmn
batch 31/32 ijhg
batch 32/32 oprs
clusters: 74552

consolidating clusters...
batch 1/16 xyzw
batch 2/16 quva
batch 3/16 kiox
batch 4/16 nghp
batch 5/16 abcd
batch 6/16 cdef
batch 7/16 klmn
batch 8/16 ijhg
batch 9/16 oprs
batch 10/16 xyzw
batch 11/16 quva
batch 12/16 kiox
batch 13/16 nghp
batch 14/16 abcd
batch 15/16 cdef
batch 16/16 klmn
clusters: 74552

reducing clusters...
clusters: 17622

Cycle 2

accumulating clusters...
batch 1/32 ijhg
batch 2/32 oprs
bat

### Model stats

In [20]:
print(cxnn.cluster_count())
print(sorted(cxnn.point_stats(), key=lambda x: x[0]))
# print(cxnn.point_stats())
marks

17287
[(0, 0), (0, 2), (0, 0), (0, 2), (0, 1), (0, 0), (0, 0), (0, 1), (0, 2), (0, 0), (0, 0), (0, 0), (0, 0), (0, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 2), (0, 0), (0, 0), (0, 0), (0, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 1), (0, 4), (0, 3), (0, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 2), (0, 14), (0, 0), (0, 2), (0, 2), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 3), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 3), (0, 1), (0, 0), (0, 0), (0, 1), (0, 0), (0, 0), (0, 1), (0, 0), (0, 1), (0, 0), (0, 0), (0, 1), (0, 0), (0, 2), (0, 0), (0, 1), (0, 0), (0, 2), (0, 0), (0, 2), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 2), (0, 1), (0, 0), (0, 0), (0, 16), (0, 12), (0, 3), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 1), (0, 0), (0, 1), (0, 3), (0, 3), (0, 9), (0, 0), (0, 0), (0, 0), (0, 0), (0, 3), (0, 0), (0, 0), (0, 0), (0, 3), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 4), (0, 4), (0, 0), (0, 0)

{(0,): 'bel',
 (1,): 'blg',
 (2,): 'eng',
 (3,): 'epo',
 (4,): 'jbo',
 (5,): 'pol',
 (6,): 'rus',
 (7,): 'ukr'}

In [24]:
print(cxnn.cluster_count())
pprint(cxnn.cluster_len_stats())
pprint(cxnn.cluster_activity_stats())

15928
{5: 643, 6: 3143, 7: 9315, 8: 2150, 9: 505, 10: 137, 11: 32, 12: 3}
{0: 15928}


In [25]:
cxnn.cluster_consolidated_stats()

{0: 7711, 1: 7397, 2: 468, 3: 162, 4: 85, 5: 49, 6: 35, 7: 14, 8: 7}

In [ ]:
def dict_value(d: dict, n=0) -> object:
    i = 0
    diter = iter(d)
    while i <= n:
        key = next(diter)
        i += 1
    return d[key]


point = cxnn.point_objects[8]
cluster_idx = 0
cluster = point.cluster_objects[cluster_idx]
pprint(cluster.stats)
print(sum(cluster.stats.values()))
# pprint(cluster.component_stats())
# pprint(cluster.bit_rate())

print(sorted(cluster.component_stats().items(), key=lambda x: x[1], reverse=True))
print(point.cluster_masks)
print(np.where(cluster.bit_mask > 0)[0])
print(point.watch_bits)
print(point.cluster_masks[cluster_idx])
print(len(point.cluster_masks), len(point.cluster_objects))

# parts = cluster.component_s
# enthropy = 0.0
# for part in parts.values():
#     enthropy += -part * math.log2(part)
# print(enthropy)

In [ ]:
cluster_with_stats = None
for wp in cxnn.point_objects:
    for cluster in wp.cluster_objects:    
        if cluster.stats:
            cluster_with_stats = cluster
            break
    else: 
        continue
    break    

if cluster_with_stats:
    print(cluster_with_stats.stats, cluster_with_stats.consolidated)
else:
    print('nothing to print')
    
    

### Save-load model

In [11]:
import pickle

def save_cxnn(cxnn: ContextNN, file_name: str):
    with open(file_name, 'wb') as f:
        pickle.dump(cxnn, f)

def load_cxnn(file_name: str) -> ContextNN:
    with open(file_name, 'rb') as f:
        cxnn = pickle.load(f)
        return cxnn
    
def save_notes(notes: BitNotes, file_name: str):
    with open(file_name, 'wb') as f:
        pickle.dump(notes, f)

def load_notes(file_name: str) -> BitNotes:
    with open(file_name, 'rb') as f:
        notes = pickle.load(f)
        return notes



In [26]:
save_cxnn(cxnn, './data/cxnn_langs.pickle')

In [ ]:
# cxnn = load_cxnn('./data/cxnn_langs.pickle')

In [ ]:
save_cxnn(cxnn, './data/cxnn_monotone.pickle')

In [9]:
save_cxnn(cxnn, './data/cxnn_probs.pickle')

In [12]:
save_cxnn(cxnn, './data/cxnn_dna.pickle')